# **CS 351 Lab 2**

## Introduction to search in AI: Problem spaces, states, and goals


## **Student Details:**

### Muhammad Abdullah - 2022323

### **Task Description:**

In this game, I started with an existing Treasure Hunt game and made some changes to adapt it. Here's a breakdown of what I did:

Game Objective:
The player (represented by 'P') needs to reach the exit ('E') while avoiding being caught by the enemy ('C'). The game is played on a grid filled with obstacles ('X'), which block movement.

Key Changes and Features:
Grid Creation:

I created a 10x10 grid where the player, enemy, exit, and obstacles are placed. The obstacles take up 20% of the grid, randomly positioned, making the game more challenging.
Player and Enemy Positions:

The player always starts at the top-left corner (0,0), and the exit is placed at the bottom-right corner.
The enemy is randomly placed on the grid, but it can't start at the same position as the player or exit.
Player Movement:

I let the player move using the W (up), A (left), S (down), and D (right) keys. If the player tries to move into an obstacle or off the grid, the move is invalid.
*Enemy AI (A Algorithm)**:

I changed the enemy's behavior by implementing the A algorithm*. The enemy uses this to find the shortest path to the player’s current position.
The A* algorithm uses Manhattan distance as the heuristic and makes the enemy smarter, as it strategically moves towards the player by calculating the most efficient route around obstacles.
Game Loop:

The game continues in a loop where the player makes a move and then the enemy follows using the A* algorithm. The game ends when the player reaches the exit or is caught by the enemy.
Visuals:

I added a simple grid display to make it easier to visualize the game. After each move, the grid is reprinted to show the current positions of the player, enemy, obstacles, and exit.
By integrating the A* algorithm, I made the enemy smarter, adding more strategy and challenge to the game compared to the original Treasure Hunt version.

### **Code:**

In [ ]:
import heapq
import random
import os

# Clear the console (works on Windows and Unix)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# Define the grid size
GRID_SIZE = 10

# Create the grid
def create_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]
    return grid

# Place the player, enemy, exit, and obstacles on the grid
def initialize_game(grid):
    size = len(grid)

    # Place the player ('P')
    player_pos = (0, 0)
    grid[player_pos[0]][player_pos[1]] = 'P'

    # Place the exit ('E') at the bottom-right corner
    exit_pos = (size - 1, size - 1)
    grid[exit_pos[0]][exit_pos[1]] = 'E'

    # Place the enemy ('C') at a random position not occupied by player or exit
    while True:
        enemy_pos = (random.randint(0, size - 1), random.randint(0, size - 1))
        if enemy_pos != player_pos and enemy_pos != exit_pos:
            grid[enemy_pos[0]][enemy_pos[1]] = 'C'
            break

    # Add obstacles ('X')
    num_obstacles = int(size * size * 0.2)  # 20% of the grid cells are obstacles
    obstacles = set()
    while len(obstacles) < num_obstacles:
        obstacle_pos = (random.randint(0, size - 1), random.randint(0, size - 1))
        if obstacle_pos not in [player_pos, enemy_pos, exit_pos] and grid[obstacle_pos[0]][obstacle_pos[1]] == ' ':
            grid[obstacle_pos[0]][obstacle_pos[1]] = 'X'
            obstacles.add(obstacle_pos)

    return player_pos, enemy_pos, exit_pos

# Check if a position is within the grid and not blocked
def is_valid_position(grid, pos):
    size = len(grid)
    x, y = pos
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'X'

# Heuristic function for A* (Manhattan distance)
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* algorithm to find the shortest path
def a_star(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_score = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            break

        neighbors = [
            (current[0] - 1, current[1]),  # Up
            (current[0] + 1, current[1]),  # Down
            (current[0], current[1] - 1),  # Left
            (current[0], current[1] + 1)   # Right
        ]

        for neighbor in neighbors:
            if is_valid_position(grid, neighbor):
                tentative_g_score = g_score[current] + 1

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score, neighbor))

    # Reconstruct path
    path = []
    current = goal
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path

# Print the grid
def print_grid(grid):
    size = len(grid)
    print('-' * (size * 4 + 1))
    for row in grid:
        print('| ' + ' | '.join(row) + ' |')
        print('-' * (size * 4 + 1))

# Game loop
def game_loop():
    grid = create_grid(GRID_SIZE)
    player_pos, enemy_pos, exit_pos = initialize_game(grid)

    while True:
        clear_console()
        print("Objective: Reach the exit 'E' without being caught by the enemy 'C'.")
        print_grid(grid)

        # Check for win or lose conditions
        if player_pos == exit_pos:
            print("Congratulations! You've reached the exit safely.")
            break
        if player_pos == enemy_pos:
            print("Oh no! The enemy has caught you.")
            break

        # Player's turn
        move = input("Move (W/A/S/D): ").upper()
        dx, dy = 0, 0
        if move == 'W':
            dx, dy = -1, 0
        elif move == 'S':
            dx, dy = 1, 0
        elif move == 'A':
            dx, dy = 0, -1
        elif move == 'D':
            dx, dy = 0, 1
        else:
            print("Invalid move. Use W/A/S/D keys.")
            continue

        new_player_pos = (player_pos[0] + dx, player_pos[1] + dy)
        if is_valid_position(grid, new_player_pos):
            grid[player_pos[0]][player_pos[1]] = ' '  # Clear old position
            player_pos = new_player_pos
            grid[player_pos[0]][player_pos[1]] = 'P'  # Update new position
        else:
            print("You can't move there!")
            continue

        # Enemy's turn using A* to find the shortest path to the player's new position
        path = a_star(grid, enemy_pos, player_pos)

        if path:
            grid[enemy_pos[0]][enemy_pos[1]] = ' '  # Clear old enemy position
            enemy_pos = path[0]  # Move enemy one step along the path
            grid[enemy_pos[0]][enemy_pos[1]] = 'C'  # Update enemy's position
        else:
            print("Enemy can't reach you!")

# Run the game
if __name__ == "__main__":
    game_loop()
